In [2]:
import boto3
import zipfile
import json
import io
import pandas as pd
import ast

In [13]:
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3',
                  endpoint_url=s3_host,
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key)

buntain = 'buntain'

In [14]:
df = pd.read_csv('~/Coding/buntain/retweet_data/All_user_dict.csv')

user_name_dict = df.set_index('userid')['twitterid'].to_dict()
user_name_dict = {key: ast.literal_eval(value) for key, value in user_name_dict.items()}
print(len(user_name_dict))

74793


In [21]:
def process(folder_prefix, user_name_dict):
    response = s3.list_objects_v2(Bucket=buntain, Prefix=folder_prefix)
    i = 0
    bad_userid = 0
    files = []
    for obj in response.get('Contents', []):
        i += 1
        object_key = obj['Key']
        if object_key.endswith('.zip'):
            files.append(object_key)
            # print(f"Processing ZIP file: {object_key}")
    print(f"number of files: {i}")
    user_count_dict = dict()
    for file in files:
        zip_object = s3.get_object(Bucket=buntain, Key=file)
        zip_contents = zip_object['Body'].read()
        zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
        for file_info in zip_file.infolist():
            with zip_file.open(file_info) as json_file:
                file_name = file_info.filename
                if not file_name.endswith("-account.json"):
                    continue
                try:
                    json_data = json_file.read().decode('utf-8')
                except:
                    print("this is a text file")
                parsed_data = json.loads(json_data)
                for ind_data in parsed_data:
                    ## we can obtain the twitterids of the retweeted accounts through user_mentions
                    for acc_key in ind_data:
                        acc = ind_data[acc_key]
                        if 'accountId' in acc.keys() and 'twitterid' in acc.keys():
                            cur_user_id = acc['accountId']
                            cur_twitterid = acc['twitterid']
                            if cur_user_id not in user_name_dict:
                                user_name_dict[cur_user_id] = [cur_twitterid]
                                print(f"new user: {cur_user_id}: {cur_twitterid}")
                            elif cur_twitterid not in user_name_dict[cur_user_id]:
                                user_name_dict[cur_user_id].append(cur_twitterid)
                                print(f"multiple twitterids for: {cur_user_id}: {cur_twitterid}")
    return

In [22]:
folder_prefix_lst = ['twitter.tmrc/August_2022/TMRC14_APAC_1/', 
                     'twitter.tmrc/August_2022/TMRC14_APAC_2/', 
                     'twitter.tmrc/October_2022/TMRC15_APAC_3/']
# folder_prefix_lst = ['twitter.tmrc/October_2022/TMRC15_APAC_3/']
dict_lst = []

for folder in folder_prefix_lst:
    print(folder)
    process(folder, user_name_dict)

twitter.tmrc/August_2022/TMRC14_APAC_1/
number of files: 1000
new user: 1372375307135545346: ADaraswati
new user: 1379643181956886530: AFradella
new user: 1376389601422499841: AIksanty
new user: 1356810747343491073: AIndadari
new user: 1362239609288019969: AKasesah
new user: 1314100576360312834: AKrusnia
new user: 1414460139105439745: AMaidasanti
new user: 1445652359439290372: ANurleza
new user: 1375304129833168897: APameliana
new user: 1334686929015586816: ARopendy
new user: 1365151942582951937: ARosmalindah
new user: 1410502722357391364: ASeptianiar
new user: 1374979520663412738: ATyaslestari
new user: 1372731444888408064: AWahyuningseh
new user: 1366621319945613316: AagustinAmelia
new user: 1335778224320598017: AameliaPriska
new user: 1337291577622814721: AanwarMarwan
new user: 1365144858273456131: AbbigailZihan
new user: 1349915209809334272: AbdillahFatir2
new user: 1348558112899227651: AbdulAjie4
new user: 1351733605861474306: AbdulHaaffzz
new user: 1387604434532798468: AbdulLesma

In [24]:
len(user_name_dict)
# user_name_dict

77793

In [25]:
print(len(user_name_dict))
twitterid_df = pd.DataFrame(user_name_dict.items(), columns = ["userid", "twitterid"])
# twitterid_df
twitterid_df.to_csv("~/Coding/buntain/retweet_data/New_all_user_dict.csv", index=False)

77793
